In [ ]:
# default_exp core

# Core

> Core class to login to your DeepWord account and generate synthetic videos

In [ ]:
#exporti
import base64
import json
import os
import random
import time
from pathlib import Path
from typing import Dict, List, Union

import cv2
import pytube
import requests
import urllib3
from pytube import YouTube

from dword.utils import *
from dword.utils import URLs, _exists, TextDicts
from IPython.display import Audio
from nbdev.showdoc import show_doc

urllib3.disable_warnings()

In [ ]:
#hide
API_KEY = os.environ['API_KEY']
SECRET_KEY = os.environ['SECRET_KEY']

In [ ]:
#export
class DeepWord:
    """
    A class for logging into your DeepWord account in Python and generating videos at scale
    """
    def __init__(self, api_key: str, secret_key: str) -> None:
        """Initialize a new DeepWord object. Login to your DeepWord account to generate api keys.
        """
        self.session = requests.session()
        self.session.verify = False
        self.headers = {'api_key': api_key, 'secerat_key': secret_key, 'Content-Type': 'application/json'}
        self._authenticate_user()

    def _authenticate_user(self):
        url = URLs.validate_token_url
        response = self.session.post(url, headers=self.headers)
        output = self._process_output(response.text)
        if output['success']:
            print('login successful')
        else:
            raise ValueError('Invalid api_key or secret_key')

    @staticmethod
    def _process_output(x): return json.loads(x)

    @property
    def available_credits(self) -> int:
        """Get the number of credits available in your DeepWord account.
        """
        url = URLs.credits_url
        response = self.session.post(url, headers=self.headers)
        try:
            output = self._process_output(response.text)
            return output['available_credits']
        except:
            raise ValueError(response.text)

    def list_videos(self) -> List[Dict]:
        """Get a list of all the videos you've generated using your DeepWord account.
        """
        url = URLs.list_vids_url
        response = self.session.post(url, headers=self.headers)
        try:
            output = self._process_output(response.text)
            return output['data']
        except:
            raise ValueError(response.text)

    @property
    def _available_languages(self) -> List: return TextDicts.langs

    def _available_speakers(self, lang) -> List: return TextDicts.speakers[lang]

    def text2speech(self, text: str, language: str, speaker: str, outfile = 'text2speech.mp3') -> str:
        if language not in self._available_languages:
            raise ValueError(f'Language {language} not available. To see available languages print obj._available_languages')
        
        if speaker not in self._available_speakers(language):
            raise ValueError(f'Invalid model for language {language}. To see available models print obj._available_speakers(language)')
        
        if Path(outfile).exists():
            os.remove(f'{outfile}')
        
        code = TextDicts.lang2code[language]
        sp, gender = speaker.split(' ')

        payload='{"text":"%s","name":"%s","gender":"%s","code":"%s"}'% (text,sp,gender,code)
        url = URLs.txt2speech_url
        response = self.session.post(url, headers=self.headers,data=payload)

        try:
            decode_bytes = base64.b64decode(response.text)
            with open(outfile, "wb") as wav_file:
                wav_file.write(decode_bytes)
            print(f"Successfully generated audio file {outfile}")
            return f'{outfile}'
        except Exception as e:
            raise ValueError(response.text)

    def download_video(self, video_id: str) -> None:
        """Download one of the synthetically generated videos on your DeepWord account. 
           The video id can be found using the ``list_generated_videos()`` function. The video
           should have finished processing to be downloadable.
           Optionally, you can use download_all_videos().
        """
        url = URLs.download_vid_url + video_id
        response = self.session.get(url, headers=self.headers)
        if response.json()['status'] is False:
            raise ValueError("Video is still processing. Unable to download it at this time.")
        try:
            r = requests.get(response.json()['video_url'], stream=True)
            with open(response.json()['video_name'], 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024): 
                    if chunk: # filter out keep-alive new chunks
                        f.write(chunk)
            return (f"Successfully downloaded video {video_id}!")
        except Exception as e:
            raise ValueError(response.text)

    def download_all_videos(self, folder: Union[str, Path]  = 'downloaded_videos') -> None:
        """Download all vidoes generated with your DeepWord account. You can also pass
        a folder or nested folders where you want the vidoes to be saved.
        """
        url = URLs.list_vids_url
        path = Path().cwd()
        
        folder = Path(folder)
        folder.mkdir(parents = True, exist_ok = True)

        response = self.session.post(url, headers=self.headers)
        try:
            for item in response.json()['data']:
                r = requests.get(item['video_url'], stream=True)

                fname1 = item['title'].replace(".mp4",'')+'.mp4'
                full_path = f'{path/folder/fname1}'

                with open(full_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=1024): 
                        if chunk: # filter out keep-alive new chunks
                            f.write(chunk)
            print(f"Successfully downloaded all videos in folder {folder}!")
        except Exception as e:
            raise ValueError(response.text)

    def download_youtube_video(self, url: str, types: str = 'video', folder = 'youtube'):
        """Download a video from YouTube. You can also donwload an audio by providing
           types = 'audio'.
        """
        folder = Path(folder) / types
        folder.mkdir(exist_ok = True, parents = True)
        if types == "video":
            pytube.YouTube(url).streams.get_highest_resolution().download(folder)
            print("downloaded youtube video successfully!")
        else:
            yt = YouTube(url) 
            stream = yt.streams.filter(only_audio=True).first()
            out_file  = stream.download(folder)
            base, ext = os.path.splitext(out_file)
            new_file = base + '_audio.mp3'
            os.rename(out_file, new_file)
            print("downloaded youtube audio successfully!")
    
    def download_audio_samples(self):
        """Download all the audio samples available on the DeepWord website
        """
        folder = Path().cwd() / 'audio_samples'
        folder.mkdir(exist_ok = True)

        url = URLs.api_get_audio_sample
        response = self.session.post(url, headers=self.headers)
        try:
            for dic in self._process_output(response.text)['sample_audio_files']:
                doc = self.session.get(dic['audio_url'])
                fname = folder / (dic['title']+dic['extension'])
                f = open(fname,"wb")
                f.write(doc.content)
                f.close()
            return ("Successfully downloaded all audio samples")
        except Exception as e:
            raise ValueError(response.text)

    def download_video_actors(self):
        """Download all the video actors available on the DeepWord website.
        """
        folder = Path().cwd() / 'video_actors'
        folder.mkdir(exist_ok = True)

        url = URLs.api_get_video_actors
        response = self.session.post(url, headers=self.headers)
        try:
            for dic in self._process_output(response.text)['sample_video_files']:
                with self.session.get(dic['video_url'], stream=True) as r:
                    r.raise_for_status()
                    fname = folder / (dic['title']+dic['extension'])
                    with open(fname, 'wb') as f:
                        for chunk in r.iter_content(chunk_size=8192): 
                            f.write(chunk)
            return ("Successfully downloaded all video actors")
        except Exception as e:
            raise ValueError(response.text)

    def trim_video(self, video: Union[str, Path], start_time: int, end_time: int, outfile: Union[str, Path] = 'trimmed_video.mp4') -> None:
        """
        Trim a video in place from start (secs) to end (secs).
        For youtube videos you can use ``download_youtube_video`` before trimming them.
        """
        start_time = to_hhmmss(int(start_time)) # 0 -> 00:00:00
        end_time = to_hhmmss(int(end_time))
        if "Content-Type" in self.headers:
            self.headers.pop("Content-Type")
        url = URLs.trim_video
        payload={"start_time": start_time,"end_time":end_time }
        files = {'video': open(video,'rb')}
        response = self.session.post(url, headers=self.headers, data=payload,files=files)
        try:
            r = self.session.get(response.json()['file_url'], stream=True)
            with open(outfile, 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024): 
                    if chunk: # filter out keep-alive new chunks
                        f.write(chunk)
            print('Successfully trimmed video')
            return outfile
        except Exception as e:
            raise ValueError(response.text)

            
    def generate_video(self, video: str, audio: str, title: str = None):
        """Generate a synthetic video using a video of a person talking and the audio
           you want them to say. You can check the status of the video using
           ``list_generated_videos`` and download it using ``download_video`` or
           ``download_all_videos``
        Args:
            video (str): Video of the person you want talking.
            audio (str): Audio you want the person to say.
            title (str, optional): Optionally provide a title for the output. Defaults to 
                                   name of the video file.
        Raises:
            ValueError: If video or audio don't exist
        """
        if not _exists(video): raise ValueError(f'File not found {video}')
        if not _exists(audio): raise ValueError(f'File not found {audio}')
        payload = {}
        if title is not None:
            payload={'name': title}
        headers = {}
        headers['api_key'] = self.headers['api_key']
        headers['secerat_key'] = self.headers['secerat_key']
        url = URLs.generate_vid_url
        files = {'video_file': open(video,'rb'),'audio_file': open(audio,'rb')}
        response = self.session.post(url, headers=headers,files=files,data=payload)
        try:
            return response.json()
        except Exception as e:
            raise ValueError(e)

In [ ]:
account = DeepWord(API_KEY, SECRET_KEY)

login successful


## Account details

In [ ]:
show_doc(DeepWord.available_credits)

<h4 id="DeepWord.available_credits" class="doc_header"><code>DeepWord.available_credits</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Get the number of credits available in your DeepWord account.
        

In [ ]:
account.available_credits

888

In [ ]:
show_doc(DeepWord.list_videos)

<h4 id="DeepWord.list_videos" class="doc_header"><code>DeepWord.list_videos</code><a href="__main__.py#L38" class="source_link" style="float:right">[source]</a></h4>

> <code>DeepWord.list_videos</code>()

Get a list of all the videos you've generated using your DeepWord account.
        

In [ ]:
all_videos = account.list_videos()
all_videos[0]

{'email': 'blablabla@yopmail.com',
 'thumbnail': 'thubnail-u5mfeqpkn6uwhsn.png',
 'title': 'Noelle',
 'video_url': 'https://videos-deep-word123.s3.us-east-2.amazonaws.com/output_data/u5mfeqpkn6uwhsn.mp4',
 'video_duration': '53.952',
 'video_id': 'u5mfeqpkn6uwhsn',
 'generate_date': '2021-04-07T02:52:57.000Z',
 'output_status': 'Complete'}

## Downloading synthetically generated videos

In [ ]:
show_doc(DeepWord.download_video)

<h4 id="DeepWord.download_video" class="doc_header"><code>DeepWord.download_video</code><a href="__main__.py#L80" class="source_link" style="float:right">[source]</a></h4>

> <code>DeepWord.download_video</code>(**`video_id`**:`str`)

Download one of the synthetically generated videos on your DeepWord account. 
The video id can be found using the ``list_generated_videos()`` function. The video
should have finished processing to be downloadable.
Optionally, you can use download_all_videos().

In [ ]:
show_doc(DeepWord.download_all_videos)

<h4 id="DeepWord.download_all_videos" class="doc_header"><code>DeepWord.download_all_videos</code><a href="__main__.py#L100" class="source_link" style="float:right">[source]</a></h4>

> <code>DeepWord.download_all_videos</code>(**`folder`**:`Union`\[`str`, `Path`\]=*`'downloaded_videos'`*)

Download all vidoes generated with your DeepWord account. You can also pass
a folder or nested folders where you want the vidoes to be saved.

In [ ]:
account.download_all_videos('my_videos')

Successfully downloaded all videos in folder my_videos!


In [ ]:
!ls my_videos

0.mp4                        first_deepword_video.mp4
1.mp4                        latoo.mp4
2.mp4                        new_video.mp4
3.mp4                        twovids.mp4
4.mp4                        video_0.mp4
5.mp4                        video_1.mp4
Anna.mp4                     video_10.mp4
Berto.mp4                    video_11.mp4
Carlos.mp4                   video_12.mp4
Dalton.mp4                   video_13.mp4
Emily.mp4                    video_14.mp4
Henry.mp4                    video_15.mp4
Isaac.mp4                    video_16.mp4
James.mp4                    video_17.mp4
Julia.mp4                    video_18.mp4
Karen.mp4                    video_19.mp4
Marcus.mp4                   video_2.mp4
Mary.mp4                     video_20.mp4
Mia.mp4                      video_3.mp4
Micheal.mp4                  video_4.mp4
Noelle.mp4                   video_5.mp4
Richard.mp4                  video_6.mp4
Sam.mp4                      video_7.mp4
Trey.mp4                     vide

## Downloading sample data

In [ ]:
show_doc(DeepWord.download_video_actors)

<h4 id="DeepWord.download_video_actors" class="doc_header"><code>DeepWord.download_video_actors</code><a href="__main__.py#L163" class="source_link" style="float:right">[source]</a></h4>

> <code>DeepWord.download_video_actors</code>()

Download all the video actors available on the DeepWord website.
        

In [ ]:
account.download_video_actors()

'Successfully downloaded all video actors'

In [ ]:
!ls video_actors

Anna.mp4    Dalton.mp4  Isaac.mp4   Karen.mp4   Mia.mp4     Richard.mp4
Berto.mp4   Emily.mp4   James.mp4   Marcus.mp4  Micheal.mp4 Sam.mp4
Carlos.mp4  Henry.mp4   Julia.mp4   Mary.mp4    Noelle.mp4  Trey.mp4


In [ ]:
show_doc(DeepWord.download_audio_samples)

<h4 id="DeepWord.download_audio_samples" class="doc_header"><code>DeepWord.download_audio_samples</code><a href="__main__.py#L144" class="source_link" style="float:right">[source]</a></h4>

> <code>DeepWord.download_audio_samples</code>()

Download all the audio samples available on the DeepWord website
        

In [ ]:
account.download_audio_samples()

'Successfully downloaded all audio samples'

In [ ]:
!ls audio_samples

Amy Poehler.mp3            Mark Cuban.mp3
Boeing CEO.mp3             Martin Scorsese.mp3
Chamath Phalihapitiya.mp3  Michael Scott.mp3
Chinese Instructor.mp3     Neil deGrasse Tyson.mp3
Edward Snowden.mp3         Peyton Manning.mp3
Elizabeth Holmes.mp3       Scottie Pippen.mp3
Financial Advice.mp3       Shaq.mp3
Footbal Meme.mp3           Swim Coach.mp3
George Hotz.mp3            Tampa Bay Rays Manager.mp3
Kevin Hart.mp3             [Korean] Bong Joon Ho.mp3


## Using text2speech

One of the big advantages of using DeepWord is you don't even have to record an audio. You can use our text2speech feature to convert your text into audio. We support a **41 languages** and a bunch of speakers for each language. To see the available languages you can do

In [ ]:
account._available_languages[:5]

['arabic', 'bengali', 'chinese', 'czech', 'danish']

In [ ]:
lang = account._available_languages[0]
lang

'arabic'

In [ ]:
speakers = account._available_speakers(lang)
speakers

['ar-XA-Wavenet-A FEMALE',
 'ar-XA-Wavenet-B MALE',
 'ar-XA-Wavenet-C MALE',
 'ar-XA-Standard-A FEMALE',
 'ar-XA-Standard-B MALE',
 'ar-XA-Standard-C MALE',
 'ar-XA-Standard-D FEMALE']

In [ ]:
text = 'I want this speaker to speak this text'
account.text2speech(text, lang, speakers[0])

Successfully generated audio file text2speech.mp3


'text2speech.mp3'

In [ ]:
Audio('text2speech.mp3')

## Downloading YouTube videos

In [ ]:
show_doc(DeepWord.download_youtube_video)

<h4 id="DeepWord.download_youtube_video" class="doc_header"><code>DeepWord.download_youtube_video</code><a href="__main__.py#L126" class="source_link" style="float:right">[source]</a></h4>

> <code>DeepWord.download_youtube_video</code>(**`url`**:`str`, **`types`**:`str`=*`'video'`*, **`folder`**=*`'youtube'`*)

Download a video from YouTube. You can also donwload an audio by providing
types = 'audio'.

> Note: If you want to download both the audio and the video of a YouTube video, download the audio first and then the video otherwise the audio will overwrite the video

In [ ]:
url = 'https://youtu.be/_81yOz-ujug'

In [ ]:
# account.download_youtube_video(url, types = 'audio')

In [ ]:
# account.download_youtube_video(url)

In [ ]:
# !ls youtube/audio/

In [ ]:
# !ls youtube/video/

## Trim videos

We trim videos on our servers for you so trim videos is part of the core module and not the utils module

In [ ]:
show_doc(DeepWord.trim_video)

<h4 id="DeepWord.trim_video" class="doc_header"><code>DeepWord.trim_video</code><a href="__main__.py#L183" class="source_link" style="float:right">[source]</a></h4>

> <code>DeepWord.trim_video</code>(**`video`**:`Union`\[`str`, `Path`\], **`start_time`**:`int`, **`end_time`**:`int`, **`outfile`**:`Union`\[`str`, `Path`\]=*`'trimmed_video.mp4'`*)

Trim a video in place from start (secs) to end (secs).
For youtube videos you can use ``download_youtube_video`` before trimming them.

In [ ]:
fname = 'test_video.mp4'
display_video(fname)

In [ ]:
account.trim_video(fname, 0, 5)

Successfully trimmed video


'trimmed_video.mp4'

In [ ]:
display_video('trimmed_video.mp4')

## Creating synthetic videos

In [ ]:
show_doc(DeepWord.generate_video)

<h4 id="DeepWord.generate_video" class="doc_header"><code>DeepWord.generate_video</code><a href="__main__.py#L208" class="source_link" style="float:right">[source]</a></h4>

> <code>DeepWord.generate_video</code>(**`video`**:`str`, **`audio`**:`str`, **`title`**:`str`=*`None`*)

Generate a synthetic video using a video of a person talking and the audio
   you want them to say. You can check the status of the video using
   ``list_generated_videos`` and download it using ``download_video`` or
   ``download_all_videos``
Args:
    video (str): Video of the person you want talking.
    audio (str): Audio you want the person to say.
    title (str, optional): Optionally provide a title for the output. Defaults to 
                           name of the video file.
Raises:
    ValueError: If video or audio don't exist

In [ ]:
account.generate_video('video_actors/Anna.mp4', 'audio_samples/Amy Poehler.mp3')

{'status': True,
 'message': 'Your video has been added to the queue for processing. Please check back in 10-15 minutes',
 'url': 'https://staging.deepword.co/video/u5mpr0tkpzvmj1x'}

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_utils.ipynb.
Converted 02_tutorials.educational.ipynb.
Converted 02_tutorials.input_types.ipynb.
Converted 02_tutorials.sales.ipynb.
Converted index.ipynb.
